In [12]:
# Cell 1: Install Libraries

!pip install yfinance pandas scikit-learn streamlit
!npm install -g localtunnel # Install localtunnel utility for exposing the Colab server

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 96.6 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇
added 22 packages in 2s
⠏
⠏3 packages are looking for funding
⠏  run `npm fund` for details
⠏

In [13]:
# Cell 2: Back-end Logic (Functions)

import yfinance as yf
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
import datetime
import streamlit as st # Imported here but primarily used in Cell 3

# --- Data Acquisition Function (Step 1) ---
def get_stock_data(ticker, start_date, end_date):
    """
    Downloads historical stock data for a given ticker and date range.
    """
    try:
        # Fetch the data
        data = yf.download(ticker, start=start_date, end=end_date)

        if data.empty:
            return "Error: No data found. Check the ticker or date range.", None

        return "Success", data

    except Exception as e:
        return f"Error downloading data: {e}", None

# --- Prediction Model Function (Step 2) ---
def predict_next_day_price(stock_df):
    """
    Trains a Linear Regression model on historical data to predict the next closing price.
    """
    if stock_df is None or stock_df.empty:
        return None, "Error: Empty DataFrame provided for prediction."

    # 1. Feature Engineering: Target variable (Prediction) is the next day's price
    stock_df['Prediction'] = stock_df['Adj Close'].shift(-1)
    df_predict = stock_df.iloc[:-1]

    # 2. Define Features (X - time index) and Target (y - next day's price)
    X = np.array(df_predict.index.factorize()[0].reshape(-1, 1))
    y = np.array(df_predict['Prediction'])

    # 3. Train the Model
    lr = LinearRegression()
    lr.fit(X, y)

    # 4. Predict the Next Day (the index immediately following the last available row)
    next_index = len(stock_df)
    X_next = np.array([[next_index]])

    predicted_price = lr.predict(X_next)[0]

    return predicted_price, "Success"

In [21]:
# Cell 3: Front-end Logic and File Creation

%%writefile app.py
import yfinance as yf
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
import streamlit as st
import datetime

# --- Back-end Functions must be defined in the file for Streamlit to run ---

def get_stock_data(ticker, start_date, end_date):
    try:
        data = yf.download(ticker, start=start_date, end=end_date)
        if data.empty:
            return "Error: No data found. Check the ticker or date range.", None
        return "Success", data
    except Exception as e:
        return f"Error downloading data: {e}", None

def predict_next_day_price(stock_df):
    if stock_df is None or stock_df.empty:
        return None, "Error: Empty DataFrame provided for prediction."

    stock_df['Prediction'] = stock_df['Close'].shift(-1)
    df_predict = stock_df.iloc[:-1]

    X = np.array(df_predict.index.factorize()[0].reshape(-1, 1))
    y = np.array(df_predict['Prediction'])

    lr = LinearRegression()
    lr.fit(X, y)

    next_index = len(stock_df)
    X_next = np.array([[next_index]])

    predicted_price = lr.predict(X_next)[0]

    return predicted_price, "Success"

# --- Main Streamlit Application UI ---

def main_app():
    st.set_page_config(
        page_title="📈 Python Stock Predictor",
        layout="wide",
        initial_sidebar_state="expanded"
    )

    st.title("📈 Stock Market Monitor and Predictor (Python-Only Stack)")
    st.markdown("Enter a stock ticker and a historical date range to get data and a price prediction.")

    # --- User Input Widgets ---
    col1, col2, col3 = st.columns(3)

    with col1:
        ticker = st.text_input("Stock Ticker (e.g., AAPL, GOOGL)", "GOOGL")

    with col2:
        start_date = st.date_input("Start Date", datetime.date(2023, 1, 1))

    with col3:
        # Default end date is yesterday for practical stock data fetching
        end_date = st.date_input("End Date", datetime.date.today() - datetime.timedelta(days=1))

    st.markdown("---")

    # --- Action Button ---
    if st.button("Fetch Data and Predict Price", type="primary"):
        if start_date >= end_date:
            st.error("Error: Start Date must be before End Date.")
            return

        # 1. Back-end Call: Data Acquisition
        st.info(f"Fetching historical data for **{ticker.upper()}** from {start_date} to {end_date}...")
        status, stock_df = get_stock_data(ticker.upper(), str(start_date), str(end_date))

        if status != "Success":
            st.error(status)
            return

        # 2. Display Data
        st.subheader(f"Historical Data for {ticker.upper()}")
        st.dataframe(stock_df.tail())

        st.subheader("Closing Price History")
        st.line_chart(stock_df['Close'])

        # 3. Back-end Call: Prediction
        st.subheader("Next Day Price Prediction (Linear Regression)")

        predicted_price, model_status = predict_next_day_price(stock_df.copy())

        if model_status == "Success":
            st.success(f"The **Predicted Closing Price** for the day after {end_date} is: **${predicted_price:,.2f}**")
            st.markdown("---")
            st.markdown("> **Model Warning:** This is a basic model for demonstration. **Do not use for financial decisions.**")
        else:
            st.error(f"Prediction Error: {model_status}")

if __name__ == '__main__':
    main_app()

Overwriting app.py


In [22]:
# New Cell: Install ngrok
!pip install pyngrok


In [23]:
# Cell 4 (REPLACED): Launch with ngrok

from pyngrok import ngrok
import subprocess
import time

# 1. STOP any previous ngrok tunnels
ngrok.kill()

# 2. Authenticate ngrok (PASTE YOUR TOKEN HERE)
!ngrok config add-authtoken 35svOWkOVf5EVXUHEURG87eihUQ_3SzRaycLhw1uzbircK3nx

# 3. Start the Streamlit app in the background
process = subprocess.Popen(['streamlit', 'run', 'app.py', '--server.port', '8501'])

# 4. Create the tunnel
# We wait a moment for Streamlit to start up
time.sleep(5)
tunnel = ngrok.connect(8501)

print(f"**Your Stable Streamlit App URL is:** {tunnel.public_url}")

# Keep the cell running to keep the tunnel open
while True:
    time.sleep(1)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
**Your Stable Streamlit App URL is:** https://intangibly-unbased-miss.ngrok-free.dev


KeyboardInterrupt: 